### Unitful, see also JuliaAstro: [UnitfulAstro.jl](https://github.com/JuliaAstro/UnitfulAstro.jl)

In [ ]:
# import Pkg; Pkg.add(["Unitful", "UnitfulAstro"])
using Unitful # , UnitfulAstro

In [ ]:
u"m"

In [ ]:
typeof(u"m")

In [ ]:
1u"m"

In [ ]:
1 * u"m"

In [ ]:
u"s"

In [ ]:
typeof(u"s")

In [ ]:
u"N"

In [ ]:
typeof(u"N")

In [ ]:
u"N" * u"m"

In [ ]:
typeof(u"m" * u"N")

### Unit convert

In [ ]:
?uconvert

In [ ]:
uconvert(u"J", u"m" * u"N")

In [ ]:
1 * u"N" * u"m"

In [ ]:
typeof(1 * u"N" * u"m")

In [ ]:
uconvert(u"J", 1 * u"N" * u"m")

In [ ]:
uconvert(u"J", 1.0 * u"N" * u"m")

In [ ]:
u"erg"

In [ ]:
typeof(u"erg")

In [ ]:
u"Msun"

In [ ]:
u"foe"

In [ ]:
u"Rjup"

In [ ]:
u"Rjup_e"

In [ ]:
UnitfulAstro.Rjup_e

In [ ]:
uconvert(u"m", 1 * UnitfulAstro.Rju_e)

## [TikzGraphs](https://nbviewer.jupyter.org/github/JuliaTeX/TikzGraphs.jl/blob/master/doc/TikzGraphs.ipynb)

In [ ]:
import Pkg; Pkg.add("TikzPictures"); Pkg.add("TikzGraphs")
import Pkg; Pkg.add("LightGraphs")

using TikzGraphs
using LightGraphs
using TikzPictures

In [ ]:
g = DiGraph(4)
add_edge!(g, 1, 2)
add_edge!(g, 2, 3)
TikzGraphs.plot(g)

In [ ]:
add_edge!(g, 3, 4)
add_edge!(g, 1, 4)
TikzGraphs.plot(g)

In [ ]:
t = TikzGraphs.plot(g)

In [ ]:
TikzPictures.save(PDF("graph"), t)
TikzPictures.save(SVG("graph"), t)
TikzPictures.save(TEX("graph"), t)

## Bad code is slow, good code is, mostly, fast 

This example is basing to large extent on
[*Julia is fast*](https://github.com/JuliaComputing/JuliaBoxTutorials/blob/master/introductory-tutorials/intro-to-julia/09.%20Julia%20is%20fast.ipynb) at JuliaComputing GitHub account. There are based on [lecture](https://github.com/mitmath/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb) by Steven G. Johnson form MIT. He is creator of Fastest Fourier Transform in the Werst (FFTW).

We test speed of Julia using simple case of function summing elements of vector $V$ with $N = 10^{ 7 }$ random Floats64 from $[ 0, 1 )$.
\begin{equation}
    \textrm{sum}(V) = \sum_{ i = 1 }^{ N } V_{ i }
\end{equation}

In [ ]:
v = rand(10^7)

function sum_bad(V)
    s = zero(eltype(v))
    for v in V
        s += v
    end
    s
end

function sum_good(V)
    s = zero(eltype(V))
    for v in V
        s += v
    end
    s
end

In [ ]:
sum_bad_bench = @benchmark sum_bad($v)
sum_good_bench = @benchmark sum_good($v)

t_bad = sum_bad_bench.times / 1e6
t_good = sum_good_bench.times / 1e6

bad_dict = Dict("Minimum time" => minimum(t_bad))
bad_dict["Mean time"] = mean(t_bad)
bad_dict["Maximum time"] = maximum(t_bad)

good_dict = Dict("Minimum time" => minimum(t_good))
good_dict["Mean time"] = mean(t_good)
good_dict["Maximum time"] = maximum(t_bad)

In [ ]:
t₀ = 0.0
t₁ = bad_dict["Maximum time"] + 50


histogram(t_good, bins=4,
                 title="Good Julia vs bad Julia",
                 xlim=(t₀, t₁),
                 ylim=(-5, 100),
                 xlabel="miliseconds", ylabel="count", label="Good code")

histogram!(t_bad, bins=4,
                   label="Bad code")

In [ ]:
println("Results for sum_bad in miliseconds [ms]")
for (key, value) in sort(collect(bad_dict), by=last)
    println(rpad(key, 26, "."), lpad(round(value; digits=5), 6, "."))
end

println("\n\n\nResults for sum_good in miliseconds [ms]")
for (key, value) in sort(collect(good_dict), by=last)
    println(rpad(key, 26, "."), lpad(round(value; digits=5), 6, "."))
end

In [ ]:
# import Pkg; Pkg.add("PyCall")
using PyCall

In [ ]:
pysum = pybuiltin("sum")

pysum_bench = @benchmark pysum($v)

t_pysum = pysum_bench.times / 1e6

In [ ]:
py"""
def py_sum(V):
    s = 0.0
    
    for v in V:
        s += v

    return s
"""

sum_py = py"py_sum"

py_hand_bench = @benchmark $sum_py($v)

t_py_hand = py_hand_bench.times / 1e6

In [ ]:
t₀ = 0.0
t₁ = maximum(t_py_hand) + 100

histogram(t_good, bins=4,
                  title="Good Julia vs bad Julia vs Python build-in vs Python by hand",
                  xlim=(t₀, t₁),
                  ylim=(-5,100),
                  xlabel="miliseconds", ylabel="count", label="Good Julia")

histogram!(t_bad, bins=4,
                  label="Julia bad")

histogram!(t_pysum, bins=1,
                    label="Python build-in")

histogram!(t_py_hand, bins=1,
                      label="Python by hand")